In [36]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
import csv

In [37]:
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [38]:
df_tce = pd.read_csv('dados_pessoal_tce_mt_cpf_v_II.csv')
df_tce.head()

,Ano,ENT_CODIGO,MUN_CODIGO,Nome,cpf,Desligamento,Codigo_scan,CPF
0,2011,1121375,510125,ADAIR JOSE RIBEIRO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,496.149.***.-68
1,2011,1121375,510125,ADAIR RODRIGUES ALVES,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: 03/01/2011,1,872.145.***.-15
2,2011,1121375,510125,ADAIR SILVERIO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,696.287.***.-34
3,2011,1121375,510125,ADAO AUGUSTO DA SILVA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,784.934.***.-91
4,2011,1121375,510125,ADAO MARLON DE CARVALHO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,861.259.***.-49


In [39]:
orgao = pd.read_csv('src/municipios.csv', )
orgao.head()

,Unnamed: 0,MUN_CODIGO,Nome da Microregião,MUN_NOME,GENTILICO,ENT_CODIGO,ENT_NOME
0,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,1121375,PREFEITURA MUNICIPAL DE ARAPUTANGA
1,1,510380,Jauru,FIGUEIROPOLIS DOESTE,FIGUEIROPOLENSE,1128727,PREFEITURA MUNICIPAL DE FIGUEIROPOLIS DOESTE
2,2,510395,Jauru,GLORIA DOESTE,GLORIENSES-DO-OESTE,1126127,PREFEITURA MUNICIPAL DE GLORIA DOESTE
3,3,510450,Jauru,INDIAVAI,INDIAVAIENSE,1125004,PREFEITURA MUNICIPAL DE INDIAVAI
4,4,510500,Jauru,JAURU,JAURUENSE,1111418,PREFEITURA MUNICIPAL DE JAURU


In [40]:
df_tce_novo = df_tce. merge( orgao, on='ENT_CODIGO')
df_tce_novo.head()

,Ano,ENT_CODIGO,MUN_CODIGO_x,Nome,cpf,Desligamento,Codigo_scan,CPF,Unnamed: 0,MUN_CODIGO_y,Nome da Microregião,MUN_NOME,GENTILICO,ENT_NOME
0,2011,1121375,510125,ADAIR JOSE RIBEIRO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,496.149.***.-68,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
1,2011,1121375,510125,ADAIR RODRIGUES ALVES,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: 03/01/2011,1,872.145.***.-15,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
2,2011,1121375,510125,ADAIR SILVERIO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,696.287.***.-34,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
3,2011,1121375,510125,ADAO AUGUSTO DA SILVA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,784.934.***.-91,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
4,2011,1121375,510125,ADAO MARLON DE CARVALHO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,861.259.***.-49,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA


In [41]:
desligamento = 'Data desligamento: Inexistente'
df_tce_novo = df_tce_novo.query('Desligamento == @desligamento')
df_tce_novo

,Ano,ENT_CODIGO,MUN_CODIGO_x,Nome,cpf,Desligamento,Codigo_scan,CPF,Unnamed: 0,MUN_CODIGO_y,Nome da Microregião,MUN_NOME,GENTILICO,ENT_NOME
0,2011,1121375,510125,ADAIR JOSE RIBEIRO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,496.149.***.-68,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
2,2011,1121375,510125,ADAIR SILVERIO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,696.287.***.-34,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
3,2011,1121375,510125,ADAO AUGUSTO DA SILVA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,784.934.***.-91,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
4,2011,1121375,510125,ADAO MARLON DE CARVALHO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,861.259.***.-49,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
5,2011,1121375,510125,ADAO RODRIGUES DE FARIAS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,872.314.***.-00,0,510125,Jauru,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20355,2020,1122134,510775,PATRICIA FERREIRA MOTA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,046.695.***.-06,11,510775,Jauru,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU
20356,2020,1122134,510775,RAINE KELLE FERNANDES DA SILVA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,046.295.***.-65,11,510775,Jauru,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU
20357,2020,1122134,510775,ROBERTO ALVES FERREIRA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,016.182.***.-09,11,510775,Jauru,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU
20358,2020,1122134,510775,SAMUEL RODRIGUES ROSA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,975.792.***.-49,11,510775,Jauru,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU


In [42]:
servidores_auxilio = pd.read_csv('servidores_auxilio.csv')

In [43]:
df = df_tce_novo.merge(servidores_auxilio, on='Nome')

In [44]:
df

,Ano,ENT_CODIGO,MUN_CODIGO_x,Nome,cpf,Desligamento,Codigo_scan,CPF,Unnamed: 0_x,MUN_CODIGO_y,...,MUN_NOME,GENTILICO,ENT_NOME,Unnamed: 0_y,municipio,cpf_beneficiario,enquadramento,Cargo,Cargo inicial,Municipio
0,2011,1121375,510125,ETENOGONIO JAKS MOREIRA DIAS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,006.650.***.-01,0,510125,...,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA,150,Araputanga,***650581**,EXTRA CADUN,VIGIA,VIGIA,Araputanga
1,2012,1121375,510125,ETENOGONIO JAKS MOREIRA DIAS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,006.650.***.-01,0,510125,...,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA,150,Araputanga,***650581**,EXTRA CADUN,VIGIA,VIGIA,Araputanga
2,2013,1121375,510125,ETENOGONIO JAKS MOREIRA DIAS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,006.650.***.-01,0,510125,...,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA,150,Araputanga,***650581**,EXTRA CADUN,VIGIA,VIGIA,Araputanga
3,2014,1121375,510125,ETENOGONIO JAKS MOREIRA DIAS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,006.650.***.-01,0,510125,...,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA,150,Araputanga,***650581**,EXTRA CADUN,VIGIA,VIGIA,Araputanga
4,2015,1121375,510125,ETENOGONIO JAKS MOREIRA DIAS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,006.650.***.-01,0,510125,...,ARAPUTANGA,ARAPUTANGUENSE,PREFEITURA MUNICIPAL DE ARAPUTANGA,150,Araputanga,***650581**,EXTRA CADUN,VIGIA,VIGIA,Araputanga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,2020,1122134,510775,MARIA APARECIDA MOREIRA DOS SANTOS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,630.194.***.-63,11,510775,...,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU,235,Jauru,***415301**,CADUN NAO BOLSA,PROFESSOR DE PEDAGOGIA,PROFESSOR DE PEDAGOGIA,Salto do Céu
977,2014,1122134,510775,MARIA HELENA DA SILVA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,496.136.***.-49,11,510775,...,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU,172,Araputanga,***397811**,BOLSA FAMILIA,PROFESSOR,PROFESSOR,Salto do Céu
978,2015,1122134,510775,MARIA HELENA DA SILVA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,496.136.***.-49,11,510775,...,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU,172,Araputanga,***397811**,BOLSA FAMILIA,PROFESSOR,PROFESSOR,Salto do Céu
979,2019,1122134,510775,JULIANA RODRIGUES DE SOUZA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,051.242.***.-38,11,510775,...,SALTO DO CEU,SALTENSE,PREFEITURA MUNICIPAL DE SALTO DO CEU,116,Salto do Céu,***242521**,BOLSA FAMILIA,AUXILIAR DE SERVICOS GERAIS,AUXILIAR DE SERVICOS GERAIS,Salto do Céu


In [45]:
df["certificador_cpf_tce"] = df['CPF'].apply(lambda x: x.split('.')[1])

In [46]:
def tratamento(x):
    digitos=''
    for i in x:
        digitos = digitos+i
        
    return(digitos)

In [47]:
df['novo_cpf_beneficiario'] = df['cpf_beneficiario'].apply(lambda x: x.split('*')[3])
df['novo_cpf_beneficiario'] = df['novo_cpf_beneficiario'].apply(lambda x: list(x))
df['novo_cpf_beneficiario'] = df['novo_cpf_beneficiario'].apply(lambda x: x[0:3])
df['novo_cpf_beneficiario'] = df['novo_cpf_beneficiario'].apply(lambda x: tratamento(x))
#df['novo_cpf_beneficiario'] = df['novo_cpf_beneficiario'].
df.T

,0,1,2,3,4,5,6,7,8,9,...,971,972,973,974,975,976,977,978,979,980
Ano,2011,2012,2013,2014,2015,2011,2012,2013,2014,2015,...,2012,2013,2012,2012,2013,2020,2014,2015,2019,2020
ENT_CODIGO,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,...,1122134,1122134,1122134,1122134,1122134,1122134,1122134,1122134,1122134,1122134
MUN_CODIGO_x,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510775,510775,510775,510775,510775,510775,510775,510775,510775,510775
Nome,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,JOAO PAULO DA SILVA,JOAO PAULO DA SILVA,JOAO PAULO DA SILVA,JOAO PAULO DA SILVA,JOAO PAULO DA SILVA,...,MARIA JOSE DE OLIVEIRA,MARIA JOSE DE OLIVEIRA,MARIA APARECIDA MOREIRA DOS SANTOS,MARIA APARECIDA MOREIRA DOS SANTOS,MARIA APARECIDA MOREIRA DOS SANTOS,MARIA APARECIDA MOREIRA DOS SANTOS,MARIA HELENA DA SILVA,MARIA HELENA DA SILVA,JULIANA RODRIGUES DE SOUZA,GUIOMAR DE MEDEIROS RODRIGUES
cpf,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...
Desligamento,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,...,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente
Codigo_scan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CPF,006.650.***.-01,006.650.***.-01,006.650.***.-01,006.650.***.-01,006.650.***.-01,496.279.***.-53,496.279.***.-53,496.279.***.-53,496.279.***.-53,496.279.***.-53,...,001.453.***.-38,001.453.***.-38,630.194.***.-63,630.194.***.-63,630.194.***.-63,630.194.***.-63,496.136.***.-49,496.136.***.-49,051.242.***.-38,010.556.***.-12
Unnamed: 0_x,0,0,0,0,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
MUN_CODIGO_y,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510775,510775,510775,510775,510775,510775,510775,510775,510775,510775


In [48]:
df = df.query('certificador_cpf_tce == novo_cpf_beneficiario')
df.T

,0,1,2,3,4,26,27,30,31,32,...,952,953,954,955,966,967,968,969,979,980
Ano,2011,2012,2013,2014,2015,2014,2020,2014,2020,2016,...,2014,2014,2015,2015,2011,2012,2013,2015,2019,2020
ENT_CODIGO,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,...,1119528,1119528,1119528,1119528,1122134,1122134,1122134,1122134,1122134,1122134
MUN_CODIGO_x,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510710,510710,510710,510710,510775,510775,510775,510775,510775,510775
Nome,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,ETENOGONIO JAKS MOREIRA DIAS,ALDAIR JOSE DE CARVALHO,ALDAIR JOSE DE CARVALHO,MARILDA DE ARAUJO GOLONI,MARILDA DE ARAUJO GOLONI,MIRIAM ALVES TEIXEIRA DE ASSIS,...,ROZIANE ALVES CORREIA DE SOUZA,ROZIANE ALVES CORREIA DE SOUZA,ROZIANE ALVES CORREIA DE SOUZA,ROZIANE ALVES CORREIA DE SOUZA,LUCILENE JACINTO DE MELO,LUCILENE JACINTO DE MELO,LUCILENE JACINTO DE MELO,LUCILENE JACINTO DE MELO,JULIANA RODRIGUES DE SOUZA,GUIOMAR DE MEDEIROS RODRIGUES
cpf,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...
Desligamento,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,...,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente
Codigo_scan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CPF,006.650.***.-01,006.650.***.-01,006.650.***.-01,006.650.***.-01,006.650.***.-01,873.415.***.-91,873.415.***.-91,420.211.***.-20,420.211.***.-20,721.182.***.-15,...,847.031.***.-72,847.031.***.-72,847.031.***.-72,847.031.***.-72,002.067.***.-97,002.067.***.-97,002.067.***.-97,002.067.***.-97,051.242.***.-38,010.556.***.-12
Unnamed: 0_x,0,0,0,0,0,0,0,0,0,0,...,8,8,8,8,11,11,11,11,11,11
MUN_CODIGO_y,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510710,510710,510710,510710,510775,510775,510775,510775,510775,510775


In [49]:
df = df.drop_duplicates('CPF')
df
#df2

,Ano,ENT_CODIGO,MUN_CODIGO_x,Nome,cpf,Desligamento,Codigo_scan,CPF,Unnamed: 0_x,MUN_CODIGO_y,...,ENT_NOME,Unnamed: 0_y,municipio,cpf_beneficiario,enquadramento,Cargo,Cargo inicial,Municipio,certificador_cpf_tce,novo_cpf_beneficiario
0,2011,1121375,510125,ETENOGONIO JAKS MOREIRA DIAS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,006.650.***.-01,0,510125,...,PREFEITURA MUNICIPAL DE ARAPUTANGA,150,Araputanga,***650581**,EXTRA CADUN,VIGIA,VIGIA,Araputanga,650,650
26,2014,1121375,510125,ALDAIR JOSE DE CARVALHO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,873.415.***.-91,0,510125,...,PREFEITURA MUNICIPAL DE ARAPUTANGA,147,Araputanga,***415141**,EXTRA CADUN,CONSELHEIRO TUTELAR,CONSELHEIRO TUTELAR,Araputanga,415,415
30,2014,1121375,510125,MARILDA DE ARAUJO GOLONI,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,420.211.***.-20,0,510125,...,PREFEITURA MUNICIPAL DE ARAPUTANGA,152,Araputanga,***211461**,EXTRA CADUN,CONSELHEIRO TUTELAR,CONSELHEIRO TUTELAR,Araputanga,211,211
32,2016,1121375,510125,MIRIAM ALVES TEIXEIRA DE ASSIS,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,721.182.***.-15,0,510125,...,PREFEITURA MUNICIPAL DE ARAPUTANGA,176,Araputanga,***182962**,CADUN NAO BOLSA,PROFESSOR,PROFESSOR,Araputanga,182,182
34,2019,1121375,510125,BRUNA RODRIGUES,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,057.887.***.-39,0,510125,...,PREFEITURA MUNICIPAL DE ARAPUTANGA,155,Araputanga,***887991**,CADUN NAO BOLSA,AGENTE DE COMBATE AS ENDEMIAS,AGENTE DE COMBATE AS ENDEMIAS,Araputanga,887,887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,2011,1119528,510710,NANCY DIAS PAVAO CANDELARIA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,531.266.***.-68,8,510710,...,PREFEITURA MUNICIPAL DE SAO JOSE DOS QUATRO MA...,141,São José dos Quatro Marcos,***266297**,CADUN NAO BOLSA,AGENTE COMUNITARIO DE SAUDE - CE 01,AGENTE COMUNITARIO DE SAUDE - CE 01,São José dos Quatro Marcos,266,266
949,2011,1119528,510710,ROZIANE ALVES CORREIA DE SOUZA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,847.031.***.-72,8,510710,...,PREFEITURA MUNICIPAL DE SAO JOSE DOS QUATRO MA...,143,São José dos Quatro Marcos,***031561**,CADUN NAO BOLSA,AUXILIAR DE SERVICOS INTERNOS - CE 01,AUXILIAR DE SERVICOS INTERNOS - CE 01,São José dos Quatro Marcos,031,031
966,2011,1122134,510775,LUCILENE JACINTO DE MELO,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,002.067.***.-97,11,510775,...,PREFEITURA MUNICIPAL DE SALTO DO CEU,111,Salto do Céu,***067931**,EXTRA CADUN,AGENTE DE SAUDE,AGENTE DE SAUDE,Salto do Céu,067,067
979,2019,1122134,510775,JULIANA RODRIGUES DE SOUZA,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,Data desligamento: Inexistente,1,051.242.***.-38,11,510775,...,PREFEITURA MUNICIPAL DE SALTO DO CEU,116,Salto do Céu,***242521**,BOLSA FAMILIA,AUXILIAR DE SERVICOS GERAIS,AUXILIAR DE SERVICOS GERAIS,Salto do Céu,242,242


# unido CPFs para pesquisa no site do TCE-MT portal radar pessoas 

In [50]:
def tratamento_cpf(x):
    digitos=''
    for i in x:
        digitos = digitos+i
        
    return(digitos)

In [51]:
df["CPF_parte_1"] = df['CPF'].apply(lambda x: x.split('.')[0:2])
df["CPF_parte_1"] = df["CPF_parte_1"].apply(lambda x: tratamento_cpf(x))
df.T

,0,26,30,32,34,35,36,37,38,39,...,900,902,904,905,909,939,949,966,979,980
Ano,2011,2014,2014,2016,2019,2019,2019,2019,2019,2019,...,2014,2014,2015,2017,2017,2011,2011,2011,2019,2020
ENT_CODIGO,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,...,1125152,1125152,1125152,1125152,1125152,1119528,1119528,1122134,1122134,1122134
MUN_CODIGO_x,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775
Nome,ETENOGONIO JAKS MOREIRA DIAS,ALDAIR JOSE DE CARVALHO,MARILDA DE ARAUJO GOLONI,MIRIAM ALVES TEIXEIRA DE ASSIS,BRUNA RODRIGUES,DHAYANNE CALITA OLIVEIRA DA SILVA,EDNA PEREIRA DE OLIVEIRA,JESSICA SANTOS STEVANATO,JUDINEI DE MELO SOUZA,NORENIR OLIVEIRA LEITE MAMEDES,...,MARIA ISABEL CALHAU,NADIR RIBEIRO DE ANDRADE,ELENIR TRAVA RICO SILVEIRA,LUIZ CARLOS DE OLIVEIRA,MOISES MARCAL DA COSTA,NANCY DIAS PAVAO CANDELARIA,ROZIANE ALVES CORREIA DE SOUZA,LUCILENE JACINTO DE MELO,JULIANA RODRIGUES DE SOUZA,GUIOMAR DE MEDEIROS RODRIGUES
cpf,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...
Desligamento,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,...,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente
Codigo_scan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CPF,006.650.***.-01,873.415.***.-91,420.211.***.-20,721.182.***.-15,057.887.***.-39,061.903.***.-28,015.361.***.-35,703.961.***.-56,026.862.***.-06,021.835.***.-80,...,030.868.***.-70,000.541.***.-37,024.085.***.-13,594.207.***.-00,046.619.***.-08,531.266.***.-68,847.031.***.-72,002.067.***.-97,051.242.***.-38,010.556.***.-12
Unnamed: 0_x,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,8,8,11,11,11
MUN_CODIGO_y,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775


In [52]:
df['CPF_parte_2'] = df['cpf_beneficiario'].apply(lambda x: x.split('*')[3])
df['CPF_parte_2'] = df['CPF_parte_2'].apply(lambda x: list(x))
df['CPF_parte_2'] = df['CPF_parte_2'].apply(lambda x: x[3:6])
df['CPF_parte_2'] = df['CPF_parte_2'].apply(lambda x: tratamento(x))
df.T

,0,26,30,32,34,35,36,37,38,39,...,900,902,904,905,909,939,949,966,979,980
Ano,2011,2014,2014,2016,2019,2019,2019,2019,2019,2019,...,2014,2014,2015,2017,2017,2011,2011,2011,2019,2020
ENT_CODIGO,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,...,1125152,1125152,1125152,1125152,1125152,1119528,1119528,1122134,1122134,1122134
MUN_CODIGO_x,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775
Nome,ETENOGONIO JAKS MOREIRA DIAS,ALDAIR JOSE DE CARVALHO,MARILDA DE ARAUJO GOLONI,MIRIAM ALVES TEIXEIRA DE ASSIS,BRUNA RODRIGUES,DHAYANNE CALITA OLIVEIRA DA SILVA,EDNA PEREIRA DE OLIVEIRA,JESSICA SANTOS STEVANATO,JUDINEI DE MELO SOUZA,NORENIR OLIVEIRA LEITE MAMEDES,...,MARIA ISABEL CALHAU,NADIR RIBEIRO DE ANDRADE,ELENIR TRAVA RICO SILVEIRA,LUIZ CARLOS DE OLIVEIRA,MOISES MARCAL DA COSTA,NANCY DIAS PAVAO CANDELARIA,ROZIANE ALVES CORREIA DE SOUZA,LUCILENE JACINTO DE MELO,JULIANA RODRIGUES DE SOUZA,GUIOMAR DE MEDEIROS RODRIGUES
cpf,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...
Desligamento,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,...,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente
Codigo_scan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CPF,006.650.***.-01,873.415.***.-91,420.211.***.-20,721.182.***.-15,057.887.***.-39,061.903.***.-28,015.361.***.-35,703.961.***.-56,026.862.***.-06,021.835.***.-80,...,030.868.***.-70,000.541.***.-37,024.085.***.-13,594.207.***.-00,046.619.***.-08,531.266.***.-68,847.031.***.-72,002.067.***.-97,051.242.***.-38,010.556.***.-12
Unnamed: 0_x,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,8,8,11,11,11
MUN_CODIGO_y,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775


In [53]:
df["CPF_parte_3"] = df['CPF'].apply(lambda x: x.split('-')[1])
df.T

,0,26,30,32,34,35,36,37,38,39,...,900,902,904,905,909,939,949,966,979,980
Ano,2011,2014,2014,2016,2019,2019,2019,2019,2019,2019,...,2014,2014,2015,2017,2017,2011,2011,2011,2019,2020
ENT_CODIGO,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,...,1125152,1125152,1125152,1125152,1125152,1119528,1119528,1122134,1122134,1122134
MUN_CODIGO_x,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775
Nome,ETENOGONIO JAKS MOREIRA DIAS,ALDAIR JOSE DE CARVALHO,MARILDA DE ARAUJO GOLONI,MIRIAM ALVES TEIXEIRA DE ASSIS,BRUNA RODRIGUES,DHAYANNE CALITA OLIVEIRA DA SILVA,EDNA PEREIRA DE OLIVEIRA,JESSICA SANTOS STEVANATO,JUDINEI DE MELO SOUZA,NORENIR OLIVEIRA LEITE MAMEDES,...,MARIA ISABEL CALHAU,NADIR RIBEIRO DE ANDRADE,ELENIR TRAVA RICO SILVEIRA,LUIZ CARLOS DE OLIVEIRA,MOISES MARCAL DA COSTA,NANCY DIAS PAVAO CANDELARIA,ROZIANE ALVES CORREIA DE SOUZA,LUCILENE JACINTO DE MELO,JULIANA RODRIGUES DE SOUZA,GUIOMAR DE MEDEIROS RODRIGUES
cpf,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...
Desligamento,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,...,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente
Codigo_scan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CPF,006.650.***.-01,873.415.***.-91,420.211.***.-20,721.182.***.-15,057.887.***.-39,061.903.***.-28,015.361.***.-35,703.961.***.-56,026.862.***.-06,021.835.***.-80,...,030.868.***.-70,000.541.***.-37,024.085.***.-13,594.207.***.-00,046.619.***.-08,531.266.***.-68,847.031.***.-72,002.067.***.-97,051.242.***.-38,010.556.***.-12
Unnamed: 0_x,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,8,8,11,11,11
MUN_CODIGO_y,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775


In [54]:
def unir_cpf (x):
    for i in x[:1]:
        s = print(i)
        
    return(s)

In [55]:
#df['CPF_completo'] = df.apply(lambda x: unir_cpf(x))

In [56]:
df.T

,0,26,30,32,34,35,36,37,38,39,...,900,902,904,905,909,939,949,966,979,980
Ano,2011,2014,2014,2016,2019,2019,2019,2019,2019,2019,...,2014,2014,2015,2017,2017,2011,2011,2011,2019,2020
ENT_CODIGO,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,1121375,...,1125152,1125152,1125152,1125152,1125152,1119528,1119528,1122134,1122134,1122134
MUN_CODIGO_x,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775
Nome,ETENOGONIO JAKS MOREIRA DIAS,ALDAIR JOSE DE CARVALHO,MARILDA DE ARAUJO GOLONI,MIRIAM ALVES TEIXEIRA DE ASSIS,BRUNA RODRIGUES,DHAYANNE CALITA OLIVEIRA DA SILVA,EDNA PEREIRA DE OLIVEIRA,JESSICA SANTOS STEVANATO,JUDINEI DE MELO SOUZA,NORENIR OLIVEIRA LEITE MAMEDES,...,MARIA ISABEL CALHAU,NADIR RIBEIRO DE ANDRADE,ELENIR TRAVA RICO SILVEIRA,LUIZ CARLOS DE OLIVEIRA,MOISES MARCAL DA COSTA,NANCY DIAS PAVAO CANDELARIA,ROZIANE ALVES CORREIA DE SOUZA,LUCILENE JACINTO DE MELO,JULIANA RODRIGUES DE SOUZA,GUIOMAR DE MEDEIROS RODRIGUES
cpf,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...,https://cidadao.tce.mt.gov.br/pessoal/detalhe/...
Desligamento,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,...,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente,Data desligamento: Inexistente
Codigo_scan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CPF,006.650.***.-01,873.415.***.-91,420.211.***.-20,721.182.***.-15,057.887.***.-39,061.903.***.-28,015.361.***.-35,703.961.***.-56,026.862.***.-06,021.835.***.-80,...,030.868.***.-70,000.541.***.-37,024.085.***.-13,594.207.***.-00,046.619.***.-08,531.266.***.-68,847.031.***.-72,002.067.***.-97,051.242.***.-38,010.556.***.-12
Unnamed: 0_x,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,7,8,8,11,11,11
MUN_CODIGO_y,510125,510125,510125,510125,510125,510125,510125,510125,510125,510125,...,510682,510682,510682,510682,510682,510710,510710,510775,510775,510775


In [61]:
df.to_csv('dados_auxilio.csv')

In [121]:
radar = csv.writer(open('radar_tce.csv' ,'w'))
driver = webdriver.Firefox()
driver.get('https://radarpessoal.tce.mt.gov.br/extensions/radarpessoal/cadastro.html')
time.sleep(50)

with open('dados_auxilio.csv') as servidores:
    try:
        consulta_cpf = servidores[23], servidores[24],servidores[25]
        #print(consulta_cpf)
        driver.find_element_by_id('main')      
        driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/article/div[1]/div/div/div/div/input').click()
        digite_cpf = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/article/div[1]/div/div/div/div/input')
        digite_cpf.send_keys(Consulta_cpf)
        driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]').click()
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[4]/div/article/div[1]/div/div/div/button').click()
        time.sleep(40)
        
        dados = driver.find_element_by_id('main')

        html = dados.get_attribute("innerHTML")
        soup = BeautifulSoup(html, "html.parser")
        
        div = soup.find_all(class_="ng-binding")
        for i in div:
            texto_i = i.text

            #print(texto_i)
            if 'ABRIL' in texto_i:
                recebeu_abril = 'Sim'


            elif 'MAIO'in texto_i:
                recebeu_maio = 'Sim'


            else:
                pass
            
        if recebeu_abril == False:
            recebeu_abril = 'Não'
            
            
        elif recebeu_maio == False:
            recebeu_maio = 'Não'
            
            
        else:
            pass
        
        
        linha = [Consulta_cpf, recebeu_abril, recebeu_maio]
        radar.writerow(linha)
            
            
        

       
    except:
        print("Busca de CPF recusada")
        print("aguarde 60 segundos para uma nova busca ")
        print("ZZzzzz...")
        time.sleep(30)
        driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[5]/div/article/div[1]/div/div/div/button').click()    
        time.sleep(3)

        
radar.close()        
driver.close()        

Busca de CPF recusada
aguarde 60 segundos para uma nova busca 
ZZzzzz...


AttributeError: '_csv.writer' object has no attribute 'close'

In [116]:
def pesquisa_cpf(consulta_cpf):
    driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/article/div[1]/div/div/div/div/input').click()
    digite_cpf = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/article/div[1]/div/div/div/div/input')
    digite_cpf.send_keys(consulta_cpf)
    driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]').click()
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[4]/div/article/div[1]/div/div/div/button').click()
    time.sleep(20)
    dados_html = driver.find_element_by_id('main')
    html = dados_html.get_attribute("innerHTML")
    soup = BeautifulSoup(html, "html.parser")

    div = soup.find_all(class_="ng-binding")
    for i in div:
        texto_i = i.text

        #print(texto_i)
        if 'ABRIL' in texto_i:
            recebeu_abril = 'Sim'


        elif 'MAIO'in texto_i:
            recebeu_maio = 'Sim'


        else:
            pass

    if recebeu_abril == False:
        recebeu_abril = 'Não'


    elif recebeu_maio == False:
        recebeu_maio = 'Não'


    else:
        pass
    
    meses = [recebeu_maio, recebeu_abril]
    return(meses)


In [112]:
def html_pesquisa(dados_html):
    html = dados_html.get_attribute("innerHTML")
    soup = BeautifulSoup(html, "html.parser")

    div = soup.find_all(class_="ng-binding")
    for i in div:
        texto_i = i.text

        #print(texto_i)
        if 'ABRIL' in texto_i:
            recebeu_abril = 'Sim'


        elif 'MAIO'in texto_i:
            recebeu_maio = 'Sim'


        else:
            pass

    if recebeu_abril == False:
        recebeu_abril = 'Não'


    elif recebeu_maio == False:
        recebeu_maio = 'Não'


    else:
        pass
    
    meses = [recebeu_maio, recebeu_abril]
    return(meses)

In [117]:
dados = 'dados_auxilio.csv'
radar = csv.writer(open('radar_tce.csv' ,'w'))
driver = webdriver.Firefox()
driver.get('https://radarpessoal.tce.mt.gov.br/extensions/radarpessoal/cadastro.html')
time.sleep(50)


with open(dados) as servidores:
    leitura = csv.reader(servidores)
    next(leitura)
    
    
    for linha in leitura:
        #print(consulta_cpf)
        try:
            consulta_cpf = str(linha[24])+str(linha[25])+str(linha[26])
            print(consulta_cpf)
            time.sleep(1)
            dados_html = pesquisa_cpf(consulta_cpf)
            #meses = html_pesquisa(dados_html)


            linha = [Consulta_cpf, meses[0], meses[1]]
            print(linha)
            #radar.writerow(linha)





        except:
            print("Busca de CPF recusada")
            print("aguarde 60 segundos para uma nova busca ")
            print("ZZzzzz...")
            time.sleep(30)
            driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[5]/div/article/div[1]/div/div/div/button').click()    
            time.sleep(3)

            

radar.close()        
driver.close()   

00665058101
Busca de CPF recusada
aguarde 60 segundos para uma nova busca 
ZZzzzz...
87341514191
Busca de CPF recusada
aguarde 60 segundos para uma nova busca 
ZZzzzz...


KeyboardInterrupt: 

In [120]:
driver.get('https://radarpessoal.tce.mt.gov.br/extensions/radarpessoal/cadastro.html')
time.sleep(50)

driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/article/div[1]/div/div/div/div/input').click()
digite_cpf = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[1]/div/article/div[1]/div/div/div/div/input')
digite_cpf.send_keys(consulta_cpf)
driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]').click()
time.sleep(3)
driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div[2]/div[2]/div[4]/div/article/div[1]/div/div/div/button').click()
time.sleep(20)
dados_html = driver.find_element_by_id('main')
html = dados_html.get_attribute("innerHTML")
soup = BeautifulSoup(html, "html.parser")

div = soup.find_all(class_="ng-binding")
for i in div:
    texto_i = i.text

    #print(texto_i)
    if 'ABRIL' in texto_i:
        recebeu_abril = 'Sim'


    elif 'MAIO'in texto_i:
        recebeu_maio = 'Sim'


    else:
        pass

if recebeu_abril == False:
    recebeu_abril = 'Não'


elif recebeu_maio == False:
    recebeu_maio = 'Não'


else:
    pass

meses = [recebeu_maio, recebeu_abril]

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=39023): Max retries exceeded with url: /session/6afb7557-63c6-445b-b74b-d38fb076c874/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f0bf022f490>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [110]:
dados

'dados_auxilio.csv'